# Imports and Constants

In [ ]:
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import numpy as np
import sklearn
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import SelectKBest, mutual_info_classif, f_classif

from saving_outputs import *
from load_data import *
from masks import *
from decoding import *
from plots import *
from utility import *
from metrics import *


SEED = 0
random.seed(SEED)
classes = ['Up', 'Down', 'Right', 'Left']
nb_runs = 12
length_one_modality = nb_runs * len(classes)
subjects_ids = range(1, 24)
n_subjects = len(subjects_ids)
n_individual_perms = 1000
labels = {'vis' : np.array(classes*nb_runs), 'aud' : np.array(classes*nb_runs)}
labels_same = np.array(classes*nb_runs)

within_modal_tasks_regions = [(["vis"], ["V5_L", "V5_R"]),
                              (["vis"], ["PT_L", "PT_R"]),
                              (["aud"], ["V5_L", "V5_R"]),
                              (["aud"], ["PT_L", "PT_R"])]

cross_modal_tasks_regions = [(["vis", "aud"], ["V5_L", "V5_R"]),
                             (["vis", "aud"], ["PT_L", "PT_R"])]

# Analyses

In [ ]:
from_who = "our"   # 2 possibilities : "mohameds" or "our"
voxel_sizes = ["3"]
#radius_mask = "7"
radiuses = range(9,10)

#masks_folder="masks/"+from_who+"_masks"+"_"+voxel_size+"_radius="+radius_mask

std_scaler = sklearn.preprocessing.StandardScaler()

classifiers = {
    'svm':sklearn.svm.SVC(C=1, random_state=SEED),
    #'LR':sklearn.linear_model.LogisticRegression(random_state=SEED)
    }
classifier = classifiers["svm"]

param_grids = {
    'svm':{
        #'svm__C': [0.1, 1],
        #'svm__gamma': [1, 0.1],
        'svm__kernel': ['linear']},
    'LR':{
        'LR__C': [1]}}


In [ ]:
acc_group_combination = dict()
voxel_amounts = dict()
for voxel_size in voxel_sizes :
    acc_group_combination[voxel_size]= dict((radius,None) for radius in radiuses)
    voxel_amounts[voxel_size]= [None]*len(radiuses)

start_time = time.time()
for k in np.linspace(0.2, 1, 9):

    def k_selector(n_voxel): return int(k*n_voxel)

    for i, radius in enumerate(radiuses) :
        for voxel_size in voxel_sizes :

            maps_folder="brain_maps/"+from_who+"_maps"+"_"+voxel_size
            masks_folder="masks/"+from_who+"_masks"+"_"+str(voxel_size)+"_radius="+str(radius)
            maps_masked, masks_exist = load_full_data(subjects_ids, len(classes), nb_runs, maps_folder, masks_folder, is_from_mohamed=(from_who=="mohameds"))
            voxel_amounts[voxel_size][i] = maps_masked[0]["vis"][0]["V5_R"].shape[1]

            models = dict()
            selector = SelectKBest(k=k_selector(voxel_amounts[voxel_size][i]), score_func=f_classif)
            for name in classifiers:
                pipeline = Pipeline([('scaler', std_scaler), ('skbest', selector), (name, classifiers[name])])
                GS = GridSearchCV(pipeline, param_grids[name], cv=nb_runs-1, n_jobs=3)
                models[name] = GS

            decoder = Decoder(n_perm=n_individual_perms, models=models, n_classes=len(classes), n_splits=nb_runs, seed=SEED)
            decoder.set_masks_exist(masks_exist)

            # launch analyses
            confusion_matrixes_within = decoder.within_modality_decoding(maps_masked, labels, subjects_ids, within_modal_tasks_regions)
            confusion_matrixes_within = change_confusion_matrixes_org(confusion_matrixes_within[0], subjects_ids, models.keys())

            confusion_matrixes_cross = decoder.cross_modality_decoding(maps_masked, labels, subjects_ids, cross_modal_tasks_regions)
            confusion_matrixes_cross = change_confusion_matrixes_org(confusion_matrixes_cross[0], subjects_ids, models.keys())

            # saving results
            out_directory = "out/"+from_who+"_"+str(voxel_size)+"_radius="+str(radius)+"_"+str(classifier)+"selecting_"+str(k*100)+"%/"
            create_directory(out_directory)

            save_dicts(out_directory + "confusion_matrixes_within.csv", confusion_matrixes_within["svm"],
                    list(confusion_matrixes_within["svm"][0].keys()), subjects_ids)
            acc_within = compute_metric(out_directory, subjects_ids, {'name' : 'accuracy', 'function':accuracy}, "within", masks_exist, len(classes), ret = True)

            save_dicts(out_directory + "confusion_matrixes_cross.csv", confusion_matrixes_cross["svm"],
                    list(confusion_matrixes_cross["svm"][0].keys()), subjects_ids)

            acc_cross = compute_metric(out_directory, subjects_ids, {'name' : 'cross', 'function':accuracy}, "cross", masks_exist, len(classes), ret = True)

            within_modality_group_results = average_dicos(acc_within)
            save_dicts(out_directory + "group_scores_within.csv", [within_modality_group_results],
                    list(within_modality_group_results.keys()), [0])
            acc_group_combination[voxel_size][radius] = within_modality_group_results

            cross_modality_group_results = average_dicos(acc_cross)
            save_dicts(out_directory + "group_scores_cross.csv", [cross_modality_group_results],
                    list(cross_modality_group_results.keys()), [0])
            acc_group_combination[voxel_size][radius].update(cross_modality_group_results)


            duration = time.time()-start_time
            print(voxel_size+" mm resolution - "+str(i+1)+"/"+str(len(radiuses))+" iterations done in "+str(duration)+" seconds")

# Plot scores depending on parameters

In [ ]:
colors = {"2":"tab:green","3":"tab:orange"}
plt_directory = "plots/"+from_who+"_voxelsizes_radiuses_"+str(classifier)+"/"
create_directory(plt_directory)
plotter = Plotter(plt_directory, subjects_ids)

for i, analysis in enumerate(acc_group_combination["2"][5]):
    # faire le plot
    for voxel_size in  voxel_sizes :
        scores = [acc_group_combination[voxel_size][radius][analysis] for radius in radiuses]
        plt.plot(radiuses, scores, '-o', color = colors[voxel_size], label = voxel_size+" mm")
        plt.axhline(0.25, color="gray", alpha=0.5)
    title = plotter.generate_title("Accuracy",analysis,0)
    plotter.save(title+ ", depending on ROI radius","","mean accuracy","radius [mm]",legend=True)

    for voxel_size in  voxel_sizes :
        scores = [acc_group_combination[voxel_size][radius][analysis] for radius in radiuses]
        n_voxels = list(map(k_selector, np.array(voxel_amounts[voxel_size])))
        plt.plot(n_voxels, scores, '-o', color = colors[voxel_size], label = voxel_size+" mm")
        print("number of voxels : "+str(voxel_amounts[voxel_size]))
        print("number of voxels selected : "+str(n_voxels))
        plt.axhline(0.25, color="gray", alpha=0.5)
    title = plotter.generate_title("Accuracy",analysis,0)
    plotter.save(title+", depending on amount of voxels","","mean accuracy","voxel amount",legend=True)

# Plot score and score variance for different k's
### (voxel_size and radius kept constant)

In [7]:
percentages = np.linspace(0.2, 1, 9)

out_folders = ["out/"+from_who+"_"+str(voxel_size)+"_radius="+str(radius)+"_"+str(classifier)+"selecting_"+str(k*100)+"%/"
               for k in percentages]

for folder in out_folders:
    acc_within = compute_metric(folder, subjects_ids, {'name' : 'accuracy', 'function':accuracy}, "within", masks_exist, len(classes), ret = True)
    acc_cross = compute_metric(folder, subjects_ids, {'name' : 'accuracy', 'function':accuracy}, "cross", masks_exist, len(classes), ret = True)

    compute_accuracy_variance(folder, "within")
    compute_accuracy_variance(folder, "cross")
    
labels = [str(int(k*voxel_amounts[voxel_size][0])) for k in percentages] 

plotter = Plotter("plots/comparing_selecting_"+"_".join(labels), subjects_ids)
plotter.plot_tests_scores_from_different_folders(out_folders, labels)
plotter.plot_accuracy_var_from_different_folders(out_folders, labels)

c:\Users\simia\Documents\MasterThesis\MasterThesisGit\saving_outputs.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(dico, ignore_index=True)
c:\Users\simia\Documents\MasterThesis\MasterThesisGit\saving_outputs.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(dico, ignore_index=True)
c:\Users\simia\Documents\MasterThesis\MasterThesisGit\saving_outputs.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(dico, ignore_index=True)
c:\Users\simia\Documents\MasterThesis\MasterThesisGit\saving_outputs.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(dico, ignore_index=Tr

<Figure size 2880x720 with 0 Axes>

<Figure size 2880x720 with 0 Axes>

<Figure size 2880x720 with 0 Axes>

<Figure size 2880x720 with 0 Axes>